In [1]:
import pandas as pd
import numpy as np
import os
import time
import subprocess
import re
import random
import arff

from math import sqrt
from sklearn.model_selection import  train_test_split
from sklearn.model_selection import KFold
from sklearn.naive_bayes import MultinomialNB  
from sklearn.metrics import classification_report
from sklearn.metrics import coverage_error
from sklearn.metrics import label_ranking_loss
from sklearn.metrics import hamming_loss
from sklearn.metrics import accuracy_score
from sklearn import metrics
from sklearn.metrics import zero_one_loss
from sklearn.metrics import jaccard_similarity_score

# functions

In [2]:
def read_data(dataPath):
    # input: '/Volumes/Samsung_T5/research/data/ABC_news_data/obesity/'
    # read data
    data = pd.read_csv(os.path.join(dataPath,'X.csv'))
    label = pd.read_csv(os.path.join(dataPath,'Y.csv'))
    return data,label

def evaluation(y_pred, y_prob, y_true):
    
    coverage = coverage_error(y_true, y_prob)
    hamming = hamming_loss(y_true, y_pred)
    ranking_loss = label_ranking_loss(y_true, y_prob) 
    
    f1_macro = metrics.f1_score(y_true, y_pred, average='macro')
    f1_micro = metrics.f1_score(y_true, y_pred, average='micro')
    
    acc = 0
    for i in range(y_true.shape[0]):
        acc += jaccard_similarity_score(y_true.iloc[i,:], y_pred.iloc[i,:]) # jaccard_similarity_score
    acc = round(acc / y_true.shape[0],2)
    
    zero_one = zero_one_loss(y_true, y_pred) # 0-1 error 
    
    f1_each = metrics.f1_score(y_true, y_pred, average=None)
    
    performance = {"coverage_error":coverage,
                   "ranking_loss":ranking_loss,
                   "hamming_loss":hamming,
                   "f1_macro":f1_macro,
                   "f1_micro":f1_micro,
                   "Jaccard_Index":acc,
                   "zero_one_error":zero_one,
                   "f1_each_label":f1_each}
    return performance

def get_confusion_matrix(y_pred, y_test, column_names):
    """confusion matrix """
    confusion_matrix = pd.DataFrame(np.array(y_pred) - np.array(y_test), columns=column_names)
    pos = pd.DataFrame((np.array(y_pred) == np.array(y_test)) & (np.array(y_pred) == 1), columns=y_test.columns).sum(axis=0)
    neg = pd.DataFrame((np.array(y_pred) == np.array(y_test)) & (np.array(y_pred) == 0), columns=y_test.columns).sum(axis=0)
    for i in range(confusion_matrix.shape[1]): 
        name = confusion_matrix.iloc[:,i].name
        temp = confusion_matrix.iloc[:,i].value_counts()
        TP = pos[name]
        TN = neg[name]
        if 1 in temp.index:
            FP = temp[1]
        else:
            FP = 0
        if -1 in temp.index:
            FN = temp[-1]
        else:
            FN = 0

# read data



In [3]:
dataPath = '/Volumes/Samsung_T5/research/data/small_datasets/yeast/'
dataset = 'yeast'
data, label = read_data(dataPath) # read data

# get data information
n_label = label.shape[1]
n_attr = data.shape[1]
n_instance = data.shape[0]
avg_label_per_instance = label.sum(axis=1).mean()
avg_instance_per_label = label.sum(axis=0).mean()
# print data information
print("\n--- Data Information ---")
print("dataset:", dataPath)
print("number of label:",n_label)
print("number of attribute:",n_attr)
print("number of instance:",n_instance)
print("avgerage number of labels for an instance:",avg_label_per_instance)
print("avgerage number of positive instances for a label:",avg_instance_per_label,"the std:",sqrt(label.sum(axis=0).var()),"\n")

print("-- number of positive instances --")
print(label.sum(axis=0))


--- Data Information ---
dataset: /Volumes/Samsung_T5/research/data/small_datasets/yeast/
number of label: 14
number of attribute: 103
number of instance: 2417
avgerage number of labels for an instance: 4.237070748862226
avgerage number of positive instances for a label: 731.5 the std: 546.4594784048141 

-- number of positive instances --
Class1      762
Class2     1038
Class3      983
Class4      862
Class5      722
Class6      597
Class7      428
Class8      480
Class9      178
Class10     253
Class11     289
Class12    1816
Class13    1799
Class14      34
dtype: int64


In [4]:
cooccurrence_matrix = label.T.dot(label)
np.fill_diagonal(cooccurrence_matrix.values, 0)
#cooccurrence_matrix.to_csv('/Users/jiangjunhao/Desktop/cooccurrence_matrix.csv', index=False)
cooccurrence_matrix

,Class1,Class2,Class3,Class4,Class5,Class6,Class7,Class8,Class9,Class10,Class11,Class12,Class13,Class14
Class1,0,614,151,107,92,167,99,115,50,43,55,505,502,2
Class2,614,0,554,174,154,199,115,160,98,130,118,718,715,3
Class3,151,554,0,594,119,97,70,115,79,128,105,755,753,34
Class4,107,174,594,0,329,48,29,45,23,40,40,740,739,34
Class5,92,154,119,329,0,441,115,130,43,41,45,608,608,6
Class6,167,199,97,48,441,0,271,227,42,56,109,476,463,1
Class7,99,115,70,29,115,271,0,363,29,28,79,365,352,1
Class8,115,160,115,45,130,227,363,0,146,34,62,372,371,2
Class9,50,98,79,23,43,42,29,146,0,54,17,96,96,1
Class10,43,130,128,40,41,56,28,34,54,0,216,177,175,0


# Binary Relevance 

## BR using naive Bayes classifier

In [5]:
def naiveBayes_multi_label_training(X_train, y_train):
    start = time.time()
    
    n_label = y_train.shape[1]
    classifier_list = [MultinomialNB() for i in range(n_label)]
    for i in range(n_label):
        classifier_list[i].fit(X_train,y_train.iloc[:,i])
    
    end = time.time()
    training_time = end-start
    
    return classifier_list, training_time

def naiveBayes_multi_label_testing(X_test, n_label, classifier_list):
    y_predict = pd.DataFrame()
    y_prob = pd.DataFrame()
    
    start = time.time()
    
    for i in range(n_label):
        y_predict_i = classifier_list[i].predict(X_test)
        y_predict = pd.concat([y_predict, pd.DataFrame(y_predict_i)],axis=1)
        
        y_predict_prob_i = classifier_list[i].predict_proba(X_test)[:,1]
        y_prob = pd.concat([y_prob, pd.DataFrame(y_predict_prob_i)],axis=1)
        
    end = time.time()
    testing_time = end-start
        
    return y_predict, y_prob, testing_time

def BR_test(data, label, dataPath, random_state=3071980):
    
    # data set information
    n_label = label.shape[1]
    n_attr = data.shape[1]
    n_instance = data.shape[0]
    avg_label_per_instance = label.sum(axis=1).mean()
    
    # split training and test data set
    X_train, X_test, y_train, y_test = train_test_split(data, label, test_size=0.5, random_state=random_state)
    
    print("-- test index --")
    print(X_test.index)
    
    # training
    classifier_list, training_time = naiveBayes_multi_label_training(X_train, y_train)
    
    # testing
    y_predict, y_prob, testing_time = naiveBayes_multi_label_testing(X_test, n_label, classifier_list)
    
    # evaluation
    performance = evaluation(y_predict, y_prob, y_test)
    
    # print data information
    print("--- Data Information ---")
    print("dataset:", dataPath)
    print("number of label:",n_label)
    print("number of attribute:",n_attr)
    print("number of instance:",n_instance,"\n")
    
    # get confusion matrix
    get_confusion_matrix(y_predict, y_test, y_test.columns)
    
    # print performance
    print("--- Performance ---")
    for key, value in performance.items():
        if key == "f1_each_label":
            print("\n- f1 for each label -")
            for i in range(n_label):
                print("label_"+label.columns[i],"=",round(value[i],2))
        elif key == "coverage_error":
            print(key,'=',round(value,2),"( avg_label_per_instance =",round(avg_label_per_instance,2),")")
        else:
            print(key,'=',round(value,2))
            
# two fold cross-validation
def two_fold_BR_test(data, label, dataPath, n_iter=5, random_state=3071980):
    
    # data set information
    n_label = label.shape[1]
    n_attr = data.shape[1]
    n_instance = data.shape[0]
    avg_label_per_instance = label.sum(axis=1).mean()
    
    # 2-fold cross validatiom
    KF=KFold(n_splits=2, shuffle=True, random_state=random_state)
    i = 0
    
    performance = {}
    for train_index,test_index in KF.split(data):
        i += 1
        
        X_train,X_test=data.iloc[train_index,:],data.iloc[test_index,:]
        y_train,y_test=label.iloc[train_index,:],label.iloc[test_index,:]
        
        print("--- kfold time="+str(i)+" ---")
        # training
        classifier_list, training_time = naiveBayes_multi_label_training(X_train, y_train)
        # testing
        y_predict, y_prob, testing_time = naiveBayes_multi_label_testing(X_test, n_label, classifier_list)
        
        # evaluation
        if performance == {}:
            performance = evaluation(y_predict, y_prob, y_test)
        else:
            performance_i = evaluation(y_predict, y_prob, y_test)
            for key, value in performance_i.items():
                performance[key] = (performance[key] + value)/2
            else:
                performance[key] = value
    
    # print data information
    print("\n--- Data Information ---")
    print("dataset:", dataPath)
    print("number of label:",n_label)
    print("number of attribute:",n_attr)
    print("number of instance:",n_instance,"\n")
    
    # print performance
    print("--- 2 fold cross-validation Performance ---")
    for key, value in performance.items():
        if key == "f1_each_label":
            continue
        elif key == "coverage_error":
            print(key,'=',round(value,2),"( avg_label_per_instance =",round(avg_label_per_instance,2),")")
        else:
            print(key,'=',round(value,2))

In [8]:
# read data
data, label = read_data(dataPath)

# train - test
print("------ Binary Relevance using Naive Bayes ------")
BR_test(data, label, dataPath,307190)

#print("")
print("------ two_fold Binary Relevance using Naive Bayes ------")
two_fold_BR_test(data, label, dataPath,307190)

------ Binary Relevance using Naive Bayes ------
-- test index --
Int64Index([1922, 2019, 1276, 2123,  700, 1114,   28, 2386,  406, 1129,
            ...
            1424,  627,  805, 1659, 1850,  988, 2373,  293, 2012,  531],
           dtype='int64', length=1209)
--- Data Information ---
dataset: /Volumes/Samsung_T5/research/data/small_datasets/yeast/
number of label: 14
number of attribute: 103
number of instance: 2417 

--- Performance ---
coverage_error = 8.04 ( avg_label_per_instance = 4.24 )
ranking_loss = 0.21
hamming_loss = 0.24
f1_macro = 0.38
f1_micro = 0.58
Jaccard_Index = 0.76
zero_one_error = 0.87

- f1 for each label -
label_Class1 = 0.43
label_Class2 = 0.56
label_Class3 = 0.57
label_Class4 = 0.49
label_Class5 = 0.5
label_Class6 = 0.33
label_Class7 = 0.28
label_Class8 = 0.29
label_Class9 = 0.09
label_Class10 = 0.06
label_Class11 = 0.02
label_Class12 = 0.8
label_Class13 = 0.8
label_Class14 = 0.06
------ two_fold Binary Relevance using Naive Bayes ------
--- kfold time=1 -

In [28]:
# data set information
n_label = label.shape[1]
n_attr = data.shape[1]
n_instance = data.shape[0]
avg_label_per_instance = label.sum(axis=1).mean()

# split training and test data set
# X_train, X_test, y_train, y_test = train_test_split(data, label, test_size=0.5, random_state=random_state)
X_train = data[data.index.isin(index)==False]
X_test = data[data.index.isin(index)==True]
y_train = label[label.index.isin(index)==False]
y_test = label[label.index.isin(index)==True]

print("-- test index --")
print(X_test.index)

# training
print("--- start training ---\n")
classifier_list, training_time = naiveBayes_multi_label_training(X_train, y_train)

# testing
print("--- start testing ---\n")
y_predict, y_prob, testing_time = naiveBayes_multi_label_testing(X_test, n_label, classifier_list)

# get confusion matrix
get_confusion_matrix(y_predict, y_test, y_test.columns)
    
# evaluation
performance = evaluation(y_predict, y_prob, y_test)


# print data information
print("--- Data Information ---")
print("dataset:", dataPath)
print("number of label:",n_label)
print("number of attribute:",n_attr)
print("number of instance:",n_instance,"\n")

# print performance
print("--- Performance ---")
for key, value in performance.items():
    if key == "f1_each_label":
        print("\n- f1 for each label -")
        for i in range(n_label):
            print("label_"+label.columns[i],"=",round(value[i],2))
    elif key == "coverage_error":
        print(key,'=',round(value,2),"( avg_label_per_instance =",round(avg_label_per_instance,2),")")
    else:
        print(key,'=',round(value,2))

-- test index --
Int64Index([   1,    5,    7,    8,   15,   16,   17,   21,   23,   24,
            ...
            2393, 2395, 2396, 2400, 2401, 2406, 2408, 2412, 2414, 2415],
           dtype='int64', length=1209)
--- start training ---

--- start testing ---

--- Data Information ---
dataset: /Volumes/Samsung_T5/research/data/small_datasets/yeast/
number of label: 14
number of attribute: 103
number of instance: 2417 

--- Performance ---
coverage_error = 8.15 ( avg_label_per_instance = 4.24 )
ranking_loss = 0.21
hamming_loss = 0.24
f1_macro = 0.38
f1_micro = 0.58
Jaccard_Index = 0.76
zero_one_error = 0.89

- f1 for each label -
label_Class1 = 0.45
label_Class2 = 0.56
label_Class3 = 0.59
label_Class4 = 0.5
label_Class5 = 0.47
label_Class6 = 0.32
label_Class7 = 0.3
label_Class8 = 0.3
label_Class9 = 0.09
label_Class10 = 0.1
label_Class11 = 0.04
label_Class12 = 0.79
label_Class13 = 0.79
label_Class14 = 0.0


## BR using ESKDB

In [29]:
def get_arff(word_occurrence, label_matrix, savePath): # get attributes
    for z in range(len(label_matrix.columns)):
        attributes=[(word_occurrence.columns[i],list(map(str,sorted(word_occurrence.iloc[:,i].unique())))) for i in range(len(word_occurrence.columns))]
        attributes.append(('label_'+label_matrix.columns[z],['0', '1']))

        data=[]
        i = 0
        while i < label_matrix.shape[0]:
            attr_data = [str(j) for j in list(word_occurrence.iloc[i,:])]
            label_data = [str(label_matrix.iloc[i,z])]
            row_data = attr_data+label_data
            data.append(row_data) 
            i+=1
        # set obj
        obj = {
           'description': u'',
           'relation': 'relation',
           'attributes': attributes,
           'data': data,
        }
        arff_data = arff.dumps(obj)
        w_file = open(savePath+label_matrix.columns[z]+".arff", "w")
        w_file.write(arff_data)
        w_file.close()

def run_eskdb(dataPath, resultFile, k, l, e, i):
    command = "./run_eskdb.sh "+resultFile+" "+k+" "+i+" "+l+" "+e+" "+dataPath
    subprocess.call("cd /Volumes/Samsung_T5/research/programme/research_python/", shell=True)
    print(command)
    return subprocess.call(command, shell=True)

def get_result(resultPath):

    y_pred = pd.DataFrame()
    y_true = pd.DataFrame()
    y_prob = pd.DataFrame()
    names = []
    for file in os.listdir(resultPath):
        with open(os.path.join(resultPath,file), 'r') as f:
            try:
                lines = f.readlines()
            except:
                print(file)
            else:
                names.append(file[:-4])
                pred = []
                true = []
                prob = []
                train_time_total = 0
                test_time_total = 0
                error_marco = 0
                for line in lines:
                    if line.startswith('pred'):
                        pred.append(int(re.search('pred :\t(.)',line).group(1)))
                        true.append(int(re.search('true :\t(.)',line).group(1)))
                        prob.append(float(re.search('prob :\t(.*)',line).group(1)))
                    elif line.startswith('RSME'):
                        rsme = float(re.search('RSME :\t\t(.*)',line).group(1))
                    elif line.startswith('Error'):
                        error = float(re.search('Error :\t\t(.*)',line).group(1))
                    elif line.startswith("Training time"):
                        train_time = float(re.search('Training time :\s{1,}(.*)',line).group(1))
                        train_time_total = train_time_total + train_time
                    elif line.startswith("Testing time"):
                        test_time = float(re.search('Testing time :\s{1,}(.*)',line).group(1))
                        test_time_total = test_time_total + test_time
                    elif line.startswith("["):
                        para = line
                    elif line.startswith("test0Indexes"):
                        index = list(map(int,re.search('test0Indexes: {(.*)}',line).group(1).split(', ')))

                y_pred = pd.concat([y_pred,pd.DataFrame(pred)],axis=1)
                y_true = pd.concat([y_true,pd.DataFrame(true)],axis=1)
                y_prob = pd.concat([y_prob,pd.DataFrame(prob)],axis=1)
    y_pred.columns = names
    y_true.columns = names
    y_pred.index = index
    y_true.index = index
    print(para)
    print("number of label:", y_pred.shape[1])
    print("training time:",train_time_total)
    print("testing time:",test_time_total)
    return y_pred,y_true,y_prob,index

In [30]:
# read data
data, label = read_data(dataPath)

# get arff files

savePath = "/Users/jiangjunhao/Desktop/test/"+dataset+'/'
if not os.path.exists(savePath):
    os.mkdir(savePath)
get_arff(data,label,savePath)

resultFile = dataset+'_k5_e20_i5000'
k = '5'
i = '5000'
l = '2'
e = '20'

run_eskdb(savePath, resultFile, k, l, e, i)

./run_eskdb.sh yeast_k5_e20_i5000 5 5000 2 20 /Users/jiangjunhao/Desktop/test/yeast/


0

In [31]:
resultPath = '/Volumes/Samsung_T5/research/programme/ESKDB_HDP/result/'+resultFile
y_pred,y_true,y_prob,index = get_result(resultPath)

performance = evaluation(y_pred=y_pred, y_true=y_true, y_prob=y_prob)
    
# print performance
print("--- Performance ---")
for key, value in performance.items():
    if key == "f1_each_label":
        print("\n- f1 for each label -")
        for i in range(n_label):
            print("label_"+label.columns[i],"=",round(value[i],2))
    elif key == "coverage_error":
        print(key,'=',round(value,2),"( avg_label_per_instance =",round(avg_label_per_instance,2),")")
    else:
        print(key,'=',round(value,2))

[-t, /Users/jiangjunhao/Desktop/test/yeast//Class9.arff, -S, ESKDB, -K, 5, -I, 5000, -L, 2, -E, 20, -V, -M]

number of label: 14
training time: 1375.0
testing time: 125.0


/Users/jiangjunhao/anaconda3/lib/python3.6/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


--- Performance ---
coverage_error = 7.61 ( avg_label_per_instance = 4.24 )
ranking_loss = 0.17
hamming_loss = 0.2
f1_macro = 0.33
f1_micro = 0.63
Jaccard_Index = 0.8
zero_one_error = 0.87

- f1 for each label -
label_Class1 = 0.48
label_Class2 = 0.0
label_Class3 = 0.0
label_Class4 = 0.86
label_Class5 = 0.86
label_Class6 = 0.0
label_Class7 = 0.58
label_Class8 = 0.65
label_Class9 = 0.48
label_Class10 = 0.49
label_Class11 = 0.14
label_Class12 = 0.15
label_Class13 = 0.0
label_Class14 = 0.0


# Classifier Chain

## CC using naive Bayes

In [32]:
def naiveBayes_multi_label_training(X_train, y_train):
    start = time.time()
    
    n_label = y_train.shape[1]
    
    order = random.sample(list(range(n_label)),n_label) # get orders
    
    classifier_list = [MultinomialNB() for i in range(n_label)] # create a classifier chain
    
    for i in range(n_label):
        if i == 0:
            classifier_list[i].fit(X_train,y_train.iloc[:, order[i]])
        else:
            X_train = pd.concat([X_train, y_train.iloc[:,order[i-1]]],axis=1) # put the previous label into attribute space
            classifier_list[i].fit(X_train,y_train.iloc[:,order[i]])

    end = time.time()
    training_time = end-start
    
    return classifier_list, training_time, order

def naiveBayes_multi_label_testing(X_test, n_label, classifier_list, order):
    y_predict = pd.DataFrame(index=X_test.index)
    y_prob = pd.DataFrame(index=X_test.index)
    y_true = pd.DataFrame(index=X_test.index)
    
    start = time.time()
    
    for i in range(n_label):
        y_predict_i = classifier_list[i].predict(X_test)
        y_predict = pd.concat([y_predict, pd.DataFrame(y_predict_i,index=X_test.index)],axis=1)

        y_predict_prob_i = classifier_list[i].predict_proba(X_test)[:,1]
        y_prob = pd.concat([y_prob, pd.DataFrame(y_predict_prob_i,index=X_test.index)],axis=1)

        X_test = pd.concat([X_test, pd.DataFrame(y_predict_i,index=X_test.index)],axis=1,ignore_index=True) # put the previous label into attribute space

    end = time.time()
    testing_time = end-start
        
    return y_predict, y_prob, testing_time

def CC_test(data, label, dataPath, random_state=3071980):
    
    # data set information
    n_label = label.shape[1]
    n_attr = data.shape[1]
    n_instance = data.shape[0]
    avg_label_per_instance = label.sum(axis=1).mean()
    
    # split training and test data set
    X_train, X_test, y_train, y_test = train_test_split(data, label, test_size=0.2, random_state=random_state)
    
    # training
    print("--- start training ---\n")
    classifier_list, training_time, order = naiveBayes_multi_label_training(X_train, y_train)
    
    # testing
    print("--- start testing ---\n")
    y_predict, y_prob, testing_time = naiveBayes_multi_label_testing(X_test, n_label, classifier_list, order)
    
    # evaluation
    y_test = y_test.iloc[:,order]
    performance = evaluation(y_predict, y_prob, y_test)
    
    # print data information
    print("--- Data Information ---")
    print("dataset:", dataPath)
    print("number of label:",n_label)
    print("number of attribute:",n_attr)
    print("number of instance:",n_instance,"\n")
    
    # print orders
    print("\n--- Order of the chain ---")
    print(label.columns[order])
    print("")
    
    # print performance
    print("--- Performance ---")
    for key, value in performance.items():
        if key == "f1_each_label":
            print("\n- f1 for each label -")
            for i in range(n_label):
                print("label_"+label.columns[i],"=",round(value[i],2))
        elif key == "coverage_error":
            print(key,'=',round(value,2),"( avg_label_per_instance =",round(avg_label_per_instance,2),")")
        else:
            print(key,'=',round(value,2))
            
def ECC_test(data, label, dataPath, random_state=3071980, ensemble = 5):
    
    # data set information
    n_label = label.shape[1]
    n_attr = data.shape[1]
    n_instance = data.shape[0]
    avg_label_per_instance = label.sum(axis=1).mean()
    
    # split training and test data set
    X_train, X_test, y_train, y_test = train_test_split(data, label, test_size=0.5, random_state=random_state)
    
    # ensemble
    y_pred_ensemble = pd.DataFrame(np.zeros(y_test.shape),columns=y_test.columns, index=y_test.index)
    y_prob_ensemble = pd.DataFrame(np.zeros(y_test.shape),columns=y_test.columns, index=y_test.index)
    for i in range(ensemble):
        # training
        #print("--- start training ---\n")
        classifier_list, training_time, order = naiveBayes_multi_label_training(X_train, y_train)

        # print orders
        print("Order of the chain:",label.columns[order])

        # testing
        #print("--- start testing ---\n")
        y_predict, y_prob, testing_time = naiveBayes_multi_label_testing(X_test, n_label, classifier_list, order)

        y_predict.columns = label.columns[order]
        y_prob.columns = label.columns[order]
        y_predict = y_predict[label.columns]
        y_prob = y_prob[label.columns]

        y_pred_ensemble = y_pred_ensemble + y_predict
        y_prob_ensemble = y_prob_ensemble + y_prob

    y_pred_ensemble = (((y_pred_ensemble / ensemble) >= 0.5)*1).astype('int')
    y_prob_ensemble = y_prob_ensemble / ensemble 
    
    # evaluation
    performance = evaluation(y_pred_ensemble, y_prob_ensemble, y_test)
    
    # print data information
    print("--- Data Information ---")
    print("dataset:", dataPath)
    print("number of label:",n_label)
    print("number of attribute:",n_attr)
    print("number of instance:",n_instance,"\n")
    
    # print orders
    print("\n--- Order of the chain ---")
    print(label.columns[order])
    print("")
    
    # print performance
    print("--- Performance ---")
    for key, value in performance.items():
        if key == "f1_each_label":
            print("\n- f1 for each label -")
            for i in range(n_label):
                print("label_"+label.columns[i],"=",round(value[i],2))
        elif key == "coverage_error":
            print(key,'=',round(value,2),"( avg_label_per_instance =",round(avg_label_per_instance,2),")")
        else:
            print(key,'=',round(value,2))

In [33]:
# read data
data, label = read_data(dataPath)

# train - test
print("------ Binary Relevance using Naive Bayes ------")
ECC_test(data, label, dataPath, 3071980, 1)

------ Binary Relevance using Naive Bayes ------
Order of the chain: Index(['Class7', 'Class5', 'Class11', 'Class2', 'Class4', 'Class12', 'Class14',
       'Class1', 'Class6', 'Class8', 'Class9', 'Class13', 'Class3', 'Class10'],
      dtype='object')
--- Data Information ---
dataset: /Volumes/Samsung_T5/research/data/small_datasets/yeast/
number of label: 14
number of attribute: 103
number of instance: 2417 


--- Order of the chain ---
Index(['Class7', 'Class5', 'Class11', 'Class2', 'Class4', 'Class12', 'Class14',
       'Class1', 'Class6', 'Class8', 'Class9', 'Class13', 'Class3', 'Class10'],
      dtype='object')

--- Performance ---
coverage_error = 8.41 ( avg_label_per_instance = 4.24 )
ranking_loss = 0.24
hamming_loss = 0.26
f1_macro = 0.38
f1_micro = 0.56
Jaccard_Index = 0.74
zero_one_error = 0.88

- f1 for each label -
label_Class1 = 0.5
label_Class2 = 0.59
label_Class3 = 0.56
label_Class4 = 0.49
label_Class5 = 0.43
label_Class6 = 0.31
label_Class7 = 0.31
label_Class8 = 0.28
lab

In [34]:
# data set information
n_label = label.shape[1]
n_attr = data.shape[1]
n_instance = data.shape[0]
avg_label_per_instance = label.sum(axis=1).mean()

# split training and test data set
# X_train, X_test, y_train, y_test = train_test_split(data, label, test_size=0.5, random_state=random_state)

X_train = data[data.index.isin(index)==False]
X_test = data[data.index.isin(index)==True]
y_train = label[label.index.isin(index)==False]
y_test = label[label.index.isin(index)==True]

print("-- test index --")
print(X_test.index)

# training
print("--- start training ---\n")
classifier_list, training_time, order = naiveBayes_multi_label_training(X_train, y_train)

# testing
print("--- start testing ---\n")
y_predict, y_prob, testing_time = naiveBayes_multi_label_testing(X_test, n_label, classifier_list, order)

# evaluation
y_test = y_test.iloc[:,order]
performance = evaluation(y_predict, y_prob, y_test)

# print data information
print("--- Data Information ---")
print("dataset:", dataPath)
print("number of label:",n_label)
print("number of attribute:",n_attr)
print("number of instance:",n_instance,"\n")

# print orders
print("\n--- Order of the chain ---")
print(label.columns[order])
print("")


# print performance
print("--- Performance ---")
for key, value in performance.items():
    if key == "f1_each_label":
        print("\n- f1 for each label -")
        for i in range(n_label):
            print("label_"+label.columns[i],"=",round(value[i],2))
    elif key == "coverage_error":
        print(key,'=',round(value,2),"( avg_label_per_instance =",round(avg_label_per_instance,2),")")
    else:
        print(key,'=',round(value,2))

-- test index --
Int64Index([   1,    5,    7,    8,   15,   16,   17,   21,   23,   24,
            ...
            2393, 2395, 2396, 2400, 2401, 2406, 2408, 2412, 2414, 2415],
           dtype='int64', length=1209)
--- start training ---

--- start testing ---

--- Data Information ---
dataset: /Volumes/Samsung_T5/research/data/small_datasets/yeast/
number of label: 14
number of attribute: 103
number of instance: 2417 


--- Order of the chain ---
Index(['Class14', 'Class9', 'Class10', 'Class11', 'Class8', 'Class1', 'Class7',
       'Class6', 'Class4', 'Class3', 'Class5', 'Class2', 'Class12', 'Class13'],
      dtype='object')

--- Performance ---
coverage_error = 8.25 ( avg_label_per_instance = 4.24 )
ranking_loss = 0.23
hamming_loss = 0.25
f1_macro = 0.37
f1_micro = 0.56
Jaccard_Index = 0.75
zero_one_error = 0.86

- f1 for each label -
label_Class1 = 0.0
label_Class2 = 0.09
label_Class3 = 0.11
label_Class4 = 0.1
label_Class5 = 0.21
label_Class6 = 0.46
label_Class7 = 0.2
label_Class8

## Ensemble CC using naive Bayes (E = 10)

In [35]:
# read data
data, label = read_data(dataPath)

# train - test
print("------ Binary Relevance using Naive Bayes ------")
ECC_test(data, label, dataPath, random_state=3071980, ensemble=10)

------ Binary Relevance using Naive Bayes ------
Order of the chain: Index(['Class13', 'Class14', 'Class4', 'Class5', 'Class6', 'Class1', 'Class7',
       'Class9', 'Class10', 'Class12', 'Class8', 'Class2', 'Class3',
       'Class11'],
      dtype='object')
Order of the chain: Index(['Class14', 'Class2', 'Class4', 'Class5', 'Class12', 'Class9', 'Class3',
       'Class6', 'Class13', 'Class8', 'Class11', 'Class10', 'Class7',
       'Class1'],
      dtype='object')
Order of the chain: Index(['Class3', 'Class13', 'Class10', 'Class2', 'Class11', 'Class9',
       'Class12', 'Class8', 'Class6', 'Class5', 'Class4', 'Class14', 'Class1',
       'Class7'],
      dtype='object')
Order of the chain: Index(['Class7', 'Class1', 'Class6', 'Class2', 'Class4', 'Class13', 'Class3',
       'Class12', 'Class9', 'Class11', 'Class5', 'Class14', 'Class8',
       'Class10'],
      dtype='object')
Order of the chain: Index(['Class13', 'Class3', 'Class1', 'Class2', 'Class12', 'Class4', 'Class7',
       'Class11'

In [36]:
# read data
data, label = read_data(dataPath)

ensemble=10

# data set information
n_label = label.shape[1]
n_attr = data.shape[1]
n_instance = data.shape[0]
avg_label_per_instance = label.sum(axis=1).mean()

X_train = data[data.index.isin(index)==False]
X_test = data[data.index.isin(index)==True]
y_train = label[label.index.isin(index)==False]
y_test = label[label.index.isin(index)==True]

print("-- test index --")
print(X_test.index)

# ensemble
y_pred_ensemble = pd.DataFrame(np.zeros(y_test.shape),columns=y_test.columns, index=y_test.index)
y_prob_ensemble = pd.DataFrame(np.zeros(y_test.shape),columns=y_test.columns, index=y_test.index)
for i in range(ensemble):
    # training
    #print("--- start training ---\n")
    classifier_list, training_time, order = naiveBayes_multi_label_training(X_train, y_train)

    # print orders
    print("Order of the chain:",label.columns[order])

    # testing
    #print("--- start testing ---\n")
    y_predict, y_prob, testing_time = naiveBayes_multi_label_testing(X_test, n_label, classifier_list, order)

    y_predict.columns = label.columns[order]
    y_prob.columns = label.columns[order]
    y_predict = y_predict[label.columns]
    y_prob = y_prob[label.columns]

    y_pred_ensemble = y_pred_ensemble + y_predict
    y_prob_ensemble = y_prob_ensemble + y_prob

y_pred_ensemble = (((y_pred_ensemble / ensemble) >= 0.5)*1).astype('int')
y_prob_ensemble = y_prob_ensemble / ensemble 

# evaluation
performance = evaluation(y_pred_ensemble, y_prob_ensemble, y_test)

# print data information
print("--- Data Information ---")
print("dataset:", dataPath)
print("number of label:",n_label)
print("number of attribute:",n_attr)
print("number of instance:",n_instance,"\n")

# print orders
print("\n--- Order of the chain ---")
print(label.columns[order])
print("")

# get confusion matrix
get_confusion_matrix(y_pred_ensemble, y_test, y_test.columns)

# print performance
print("--- Performance ---")
for key, value in performance.items():
    if key == "f1_each_label":
        print("\n- f1 for each label -")
        for i in range(n_label):
            print("label_"+label.columns[i],"=",round(value[i],2))
    elif key == "coverage_error":
        print(key,'=',round(value,2),"( avg_label_per_instance =",round(avg_label_per_instance,2),")")
    else:
        print(key,'=',round(value,2))

-- test index --
Int64Index([   1,    5,    7,    8,   15,   16,   17,   21,   23,   24,
            ...
            2393, 2395, 2396, 2400, 2401, 2406, 2408, 2412, 2414, 2415],
           dtype='int64', length=1209)
Order of the chain: Index(['Class8', 'Class1', 'Class6', 'Class11', 'Class5', 'Class9', 'Class2',
       'Class3', 'Class10', 'Class7', 'Class12', 'Class4', 'Class13',
       'Class14'],
      dtype='object')
Order of the chain: Index(['Class13', 'Class5', 'Class2', 'Class11', 'Class4', 'Class14', 'Class3',
       'Class1', 'Class8', 'Class6', 'Class9', 'Class10', 'Class12', 'Class7'],
      dtype='object')
Order of the chain: Index(['Class4', 'Class9', 'Class11', 'Class3', 'Class7', 'Class13', 'Class12',
       'Class10', 'Class5', 'Class2', 'Class1', 'Class6', 'Class8', 'Class14'],
      dtype='object')
Order of the chain: Index(['Class7', 'Class2', 'Class9', 'Class3', 'Class10', 'Class13', 'Class12',
       'Class8', 'Class6', 'Class14', 'Class11', 'Class1', 'Class5', '

## Ensemble CC using naive Bayes (E = 50)

In [37]:
# read data
data, label = read_data(dataPath)

# train - test
print("------ Binary Relevance using Naive Bayes ------")
ECC_test(data, label, dataPath, random_state=3071980, ensemble=50)

------ Binary Relevance using Naive Bayes ------
Order of the chain: Index(['Class6', 'Class10', 'Class4', 'Class7', 'Class2', 'Class8', 'Class11',
       'Class13', 'Class1', 'Class5', 'Class9', 'Class14', 'Class3',
       'Class12'],
      dtype='object')
Order of the chain: Index(['Class10', 'Class14', 'Class2', 'Class4', 'Class13', 'Class8', 'Class7',
       'Class11', 'Class9', 'Class3', 'Class12', 'Class5', 'Class6', 'Class1'],
      dtype='object')
Order of the chain: Index(['Class13', 'Class1', 'Class5', 'Class10', 'Class3', 'Class14', 'Class2',
       'Class6', 'Class12', 'Class7', 'Class4', 'Class11', 'Class8', 'Class9'],
      dtype='object')
Order of the chain: Index(['Class7', 'Class1', 'Class14', 'Class5', 'Class6', 'Class13', 'Class9',
       'Class2', 'Class11', 'Class8', 'Class10', 'Class3', 'Class4',
       'Class12'],
      dtype='object')
Order of the chain: Index(['Class5', 'Class10', 'Class14', 'Class8', 'Class9', 'Class7', 'Class2',
       'Class4', 'Class6', 'Cl

Order of the chain: Index(['Class5', 'Class7', 'Class10', 'Class14', 'Class8', 'Class1', 'Class11',
       'Class4', 'Class2', 'Class12', 'Class3', 'Class9', 'Class6', 'Class13'],
      dtype='object')
Order of the chain: Index(['Class6', 'Class9', 'Class1', 'Class5', 'Class7', 'Class2', 'Class12',
       'Class14', 'Class13', 'Class11', 'Class8', 'Class10', 'Class4',
       'Class3'],
      dtype='object')
Order of the chain: Index(['Class8', 'Class12', 'Class10', 'Class13', 'Class1', 'Class4',
       'Class14', 'Class11', 'Class5', 'Class7', 'Class2', 'Class6', 'Class9',
       'Class3'],
      dtype='object')
Order of the chain: Index(['Class2', 'Class5', 'Class13', 'Class9', 'Class4', 'Class11', 'Class14',
       'Class10', 'Class1', 'Class12', 'Class8', 'Class6', 'Class7', 'Class3'],
      dtype='object')
Order of the chain: Index(['Class11', 'Class12', 'Class2', 'Class5', 'Class7', 'Class9', 'Class14',
       'Class13', 'Class6', 'Class8', 'Class4', 'Class3', 'Class1', 'Class10']

In [38]:
# read data
data, label = read_data(dataPath)

ensemble=50

# data set information
n_label = label.shape[1]
n_attr = data.shape[1]
n_instance = data.shape[0]
avg_label_per_instance = label.sum(axis=1).mean()

X_train = data[data.index.isin(index)==False]
X_test = data[data.index.isin(index)==True]
y_train = label[label.index.isin(index)==False]
y_test = label[label.index.isin(index)==True]

print("-- test index --")
print(X_test.index)

# ensemble
y_pred_ensemble = pd.DataFrame(np.zeros(y_test.shape),columns=y_test.columns, index=y_test.index)
y_prob_ensemble = pd.DataFrame(np.zeros(y_test.shape),columns=y_test.columns, index=y_test.index)
for i in range(ensemble):
    # training
    #print("--- start training ---\n")
    classifier_list, training_time, order = naiveBayes_multi_label_training(X_train, y_train)

    # print orders
    print("Order of the chain:",label.columns[order])

    # testing
    #print("--- start testing ---\n")
    y_predict, y_prob, testing_time = naiveBayes_multi_label_testing(X_test, n_label, classifier_list, order)

    y_predict.columns = label.columns[order]
    y_prob.columns = label.columns[order]
    y_predict = y_predict[label.columns]
    y_prob = y_prob[label.columns]

    y_pred_ensemble = y_pred_ensemble + y_predict
    y_prob_ensemble = y_prob_ensemble + y_prob

y_pred_ensemble = (((y_pred_ensemble / ensemble) >= 0.5)*1).astype('int')
y_prob_ensemble = y_prob_ensemble / ensemble 

# evaluation
performance = evaluation(y_pred_ensemble, y_prob_ensemble, y_test)


# print data information
print("--- Data Information ---")
print("dataset:", dataPath)
print("number of label:",n_label)
print("number of attribute:",n_attr)
print("number of instance:",n_instance,"\n")

# print orders
print("\n--- Order of the chain ---")
print(label.columns[order])
print("")

# get confusion matrix
get_confusion_matrix(y_pred_ensemble, y_test, y_test.columns)

# print performance
print("--- Performance ---")
for key, value in performance.items():
    if key == "f1_each_label":
        print("\n- f1 for each label -")
        for i in range(n_label):
            print("label_"+label.columns[i],"=",round(value[i],2))
    elif key == "coverage_error":
        print(key,'=',round(value,2),"( avg_label_per_instance =",round(avg_label_per_instance,2),")")
    else:
        print(key,'=',round(value,2))

-- test index --
Int64Index([   1,    5,    7,    8,   15,   16,   17,   21,   23,   24,
            ...
            2393, 2395, 2396, 2400, 2401, 2406, 2408, 2412, 2414, 2415],
           dtype='int64', length=1209)
Order of the chain: Index(['Class4', 'Class1', 'Class5', 'Class14', 'Class11', 'Class10', 'Class8',
       'Class6', 'Class2', 'Class9', 'Class3', 'Class7', 'Class12', 'Class13'],
      dtype='object')
Order of the chain: Index(['Class3', 'Class14', 'Class10', 'Class5', 'Class6', 'Class13', 'Class4',
       'Class12', 'Class11', 'Class2', 'Class1', 'Class8', 'Class7', 'Class9'],
      dtype='object')
Order of the chain: Index(['Class11', 'Class9', 'Class1', 'Class2', 'Class10', 'Class14',
       'Class12', 'Class5', 'Class6', 'Class8', 'Class3', 'Class13', 'Class7',
       'Class4'],
      dtype='object')
Order of the chain: Index(['Class3', 'Class9', 'Class1', 'Class4', 'Class11', 'Class10', 'Class6',
       'Class8', 'Class14', 'Class7', 'Class2', 'Class5', 'Class13',
  

Order of the chain: Index(['Class3', 'Class7', 'Class4', 'Class5', 'Class2', 'Class14', 'Class10',
       'Class8', 'Class6', 'Class11', 'Class13', 'Class12', 'Class9',
       'Class1'],
      dtype='object')
Order of the chain: Index(['Class7', 'Class2', 'Class6', 'Class13', 'Class3', 'Class4', 'Class9',
       'Class5', 'Class8', 'Class14', 'Class11', 'Class1', 'Class10',
       'Class12'],
      dtype='object')
Order of the chain: Index(['Class6', 'Class10', 'Class1', 'Class14', 'Class2', 'Class4', 'Class13',
       'Class7', 'Class3', 'Class5', 'Class11', 'Class9', 'Class8', 'Class12'],
      dtype='object')
Order of the chain: Index(['Class4', 'Class13', 'Class5', 'Class2', 'Class9', 'Class14', 'Class10',
       'Class11', 'Class12', 'Class6', 'Class7', 'Class1', 'Class8', 'Class3'],
      dtype='object')
Order of the chain: Index(['Class4', 'Class5', 'Class7', 'Class8', 'Class2', 'Class14', 'Class10',
       'Class3', 'Class11', 'Class1', 'Class9', 'Class13', 'Class12',
       'C

## CC using ESKDB


In [39]:
# prepare data
def csv_to_arff(X, label_i, savePath):
    # get attributes
    attributes=[(X.columns[i],['0', '1']) for i in range(len(X.columns))]
    attributes.append(('label_'+label_i.name,['0', '1']))

    data=[]
    i = 0
    while i < len(label_i):
        attr_data = [str(j) for j in list(X.iloc[i,:])]
        label_data = [str(label_i[i])]
        row_data = attr_data+label_data
        data.append(row_data) 
        i+=1
    # set obj
    obj = {
       'description': u'',
       'relation': 'relation',
       'attributes': attributes,
       'data': data,
    }
    arff_data = arff.dumps(obj)
    w_file = open(savePath+label_i.name+".arff", "w")
    w_file.write(arff_data)
    w_file.close()

def get_arff(X, label, savePath):
    
    n_label = label.shape[1]
    # get orders
    order = random.sample(list(range(n_label)),n_label) # get orders
    
    #  get all arff files, one for each label
    for i in range(n_label):
        label_i = label.iloc[:,order[i]]
        print("--Running label:",label_i.name)
        csv_to_arff(X, label_i, savePath)
        
        label_i.name = 'label_' + label_i.name
        X = pd.concat([X, label_i], axis=1)
    print("--finished getting arff files")
    return order

def run_eskdb(label_arff, resultFile, k, l, e, i):
    command = "./run_ECC.sh "+resultFile+" "+k+" "+i+" "+l+" "+e+" "+label_arff
    subprocess.call("cd /Volumes/Samsung_T5/research/programme/research_python/", shell=True)
    #print(command)
    return subprocess.call(command, shell=True)

def get_result(resultPath):

    y_pred = pd.DataFrame()
    y_true = pd.DataFrame()
    y_prob = pd.DataFrame()
    names = []
    for file in os.listdir(resultPath):
        with open(os.path.join(resultPath,file), 'r') as f:
            try:
                lines = f.readlines()
            except:
                print(file)
            else:
                names.append(file[:-4])
                pred = []
                true = []
                prob = []
                train_time_total = 0
                test_time_total = 0
                error_marco = 0
                for line in lines:
                    if line.startswith('pred'):
                        pred.append(int(re.search('pred :\t(.)',line).group(1)))
                        true.append(int(re.search('true :\t(.)',line).group(1)))
                        prob.append(float(re.search('prob :\t(.*)',line).group(1)))
                    elif line.startswith('RSME'):
                        rsme = float(re.search('RSME :\t\t(.*)',line).group(1))
                    elif line.startswith('Error'):
                        error = float(re.search('Error :\t\t(.*)',line).group(1))
                    elif line.startswith("Training time"):
                        train_time = float(re.search('Training time :\s{1,}(.*)',line).group(1))
                        train_time_total = train_time_total + train_time
                    elif line.startswith("Testing time"):
                        test_time = float(re.search('Testing time :\s{1,}(.*)',line).group(1))
                        test_time_total = test_time_total + test_time
                    elif line.startswith("["):
                        para = line
                    elif line.startswith("test0Indexes"):
                        index = list(map(int,re.search('test0Indexes: {(.*)}',line).group(1).split(', ')))

                y_pred = pd.concat([y_pred,pd.DataFrame(pred)],axis=1)
                y_true = pd.concat([y_true,pd.DataFrame(true)],axis=1)
                y_prob = pd.concat([y_prob,pd.DataFrame(prob)],axis=1)
    y_pred.columns = names
    y_true.columns = names
    y_prob.columns = names
    print(para)
    print("number of label:", y_pred.shape[1])
    print("training time:",train_time_total)
    print("testing time:",test_time_total)
    return y_pred,y_true,y_prob,index

In [40]:
# read data
data, label = read_data(dataPath)
n_label = label.shape[1]

resultFile = dataset+'_k5_e20_i5000_CC'

# train and test on the first label
savePath = "/Users/jiangjunhao/Desktop/test/"+dataset+'/'

# data set information
n_label = label.shape[1]
n_attr = data.shape[1]
n_instance = data.shape[0]
avg_label_per_instance = label.sum(axis=1).mean()

# get orders
order = random.sample(list(range(n_label)),n_label) # get orders


## prepare data, get the arff file
for i in range(n_label):
    label_i = label.iloc[:,order[i]]
    #print("--Running label:",label_i.name)
    csv_to_arff(data, label_i, savePath)

    # run eskdb
    label_arff = os.path.join(savePath,label_i.name+'.arff')
    k = '5'
    i = '5000'
    l = '2'
    e = '20'

    run_eskdb(label_arff, resultFile, k, l, e, i)

    result = os.path.join("/Volumes/Samsung_T5/research/programme/ESKDB_HDP/result",resultFile, label_i.name+'.txt')
    with open(result, 'r') as f:
        try:
            lines = f.readlines()
        except:
            print(file)
        else:
            pred = []
            for line in lines:
                if line.startswith('pred'):
                    pred.append(int(re.search('pred :\t(.)',line).group(1)))
                elif line.startswith("test0Indexes"):
                    index = list(map(int,re.search('test0Indexes: {(.*)}',line).group(1).split(', ')))

            label.loc[index,label_i.name] = pred
            temp = label.loc[:,label_i.name]
            temp.name = 'label_'+label_i.name
            data = pd.concat([data, label.loc[:,label_i.name]],axis=1)

# get result
resultPath = '/Volumes/Samsung_T5/research/programme/ESKDB_HDP/result/'+resultFile+'/'
y_pred,y_true,y_prob,index = get_result(resultPath)
performance = evaluation(y_pred=y_pred, y_true=y_true, y_prob=y_prob)

# print orders:
print(label.columns[order])

# get confusion matrix
print("\n--- Confusion matrix ---")
get_confusion_matrix(y_pred, y_true, y_pred.columns)

# print performance
print("--- Performance ---")
for key, value in performance.items():
    if key == "f1_each_label":
        print("\n- f1 for each label -")
        for i in range(n_label):
            print("label_"+label.columns[i],"=",round(value[i],2))
    elif key == "coverage_error":
        print(key,'=',round(value,2),"( avg_label_per_instance =",round(avg_label_per_instance,2),")")
    else:
        print(key,'=',round(value,2))

[-t, /Users/jiangjunhao/Desktop/test/yeast/Class2.arff, -S, ESKDB, -K, 5, -I, 5000, -L, 2, -E, 20, -V, -M]

number of label: 14
training time: 1589.0
testing time: 438.0


/Users/jiangjunhao/anaconda3/lib/python3.6/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


Index(['Class7', 'Class9', 'Class14', 'Class13', 'Class5', 'Class12', 'Class3',
       'Class1', 'Class8', 'Class11', 'Class4', 'Class6', 'Class10', 'Class2'],
      dtype='object')

--- Confusion matrix ---
--- Performance ---
coverage_error = 8.62 ( avg_label_per_instance = 4.24 )
ranking_loss = 0.22
hamming_loss = 0.21
f1_macro = 0.36
f1_micro = 0.64
Jaccard_Index = 0.79
zero_one_error = 0.78

- f1 for each label -
label_Class1 = 0.13
label_Class2 = 0.0
label_Class3 = 0.0
label_Class4 = 0.86
label_Class5 = 0.53
label_Class6 = 0.86
label_Class7 = 0.66
label_Class8 = 0.51
label_Class9 = 0.1
label_Class10 = 0.01
label_Class11 = 0.61
label_Class12 = 0.32
label_Class13 = 0.02
label_Class14 = 0.47


## Ensemble CC using ESKDB(E=10)

In [43]:
# read data
data, label = read_data(dataPath)
n_label = label.shape[1]

# data set information
n_label = label.shape[1]
n_attr = data.shape[1]
n_instance = data.shape[0]
avg_label_per_instance = label.sum(axis=1).mean()

# ensemble
ensemble = 10

y_pred_ensemble = pd.DataFrame(np.zeros((int(label.shape[0]/2+1),label.shape[1])),columns=label.columns)
y_prob_ensemble = pd.DataFrame(np.zeros((int(label.shape[0]/2+1),label.shape[1])),columns=label.columns)
    
    
for i in range(ensemble):

    # read data
    data, label = read_data(dataPath)
    n_label = label.shape[1]

    # data set information
    n_label = label.shape[1]
    n_attr = data.shape[1]
    n_instance = data.shape[0]
    avg_label_per_instance = label.sum(axis=1).mean()

    # get orders
    order = random.sample(list(range(n_label)),n_label) # get orders

    # train and test on the first label
    savePath = "/Users/jiangjunhao/Desktop/test/"+dataset+'/'

    ## prepare data, get the arff file
    for i in range(n_label):
        label_i = label.iloc[:,order[i]]
        print("--Running label:",label_i.name)
        csv_to_arff(data, label_i, savePath)

        # run eskdb
        label_arff = os.path.join(savePath,label_i.name+'.arff')
        k = '5'
        i = '5000'
        l = '2'
        e = '20'

        run_eskdb(label_arff, resultFile, k, l, e, i)

        result = os.path.join("/Volumes/Samsung_T5/research/programme/ESKDB_HDP/result",resultFile, label_i.name+'.txt')
        with open(result, 'r') as f:
            try:
                lines = f.readlines()
            except:
                print(file)
            else:
                pred = []
                for line in lines:
                    if line.startswith('pred'):
                        pred.append(int(re.search('pred :\t(.)',line).group(1)))
                    elif line.startswith("test0Indexes"):
                        index = list(map(int,re.search('test0Indexes: {(.*)}',line).group(1).split(', ')))

                label.loc[index,label_i.name] = pred
                temp = label.loc[:,label_i.name]
                temp.name = 'label_'+label_i.name
                data = pd.concat([data, label.loc[:,label_i.name]],axis=1)

    # get result
    resultPath = '/Volumes/Samsung_T5/research/programme/ESKDB_HDP/result/'+resultFile+'/'
    y_pred,y_true,y_prob,index = get_result(resultPath)
    y_pred.columns = label.columns
    y_prob.columns = label.columns
    
    y_pred_ensemble = y_pred_ensemble + y_pred
    y_prob_ensemble = y_prob_ensemble + y_prob
    
    
y_pred_ensemble = (((y_pred_ensemble / ensemble) >= 0.5)*1).astype('int')
y_prob_ensemble = y_prob_ensemble / ensemble 

--Running label: Class11
--Running label: Class9
--Running label: Class1
--Running label: Class7
--Running label: Class10
--Running label: Class13
--Running label: Class2
--Running label: Class3
--Running label: Class14
--Running label: Class5
--Running label: Class8
--Running label: Class12
--Running label: Class4
--Running label: Class6
[-t, /Users/jiangjunhao/Desktop/test/yeast/Class2.arff, -S, ESKDB, -K, 5, -I, 5000, -L, 2, -E, 20, -V, -M]

number of label: 14
training time: 1532.0
testing time: 442.0
--Running label: Class10
--Running label: Class8
--Running label: Class3
--Running label: Class12
--Running label: Class2
--Running label: Class4
--Running label: Class6
--Running label: Class13
--Running label: Class14
--Running label: Class9
--Running label: Class11
--Running label: Class1
--Running label: Class5
--Running label: Class7
[-t, /Users/jiangjunhao/Desktop/test/yeast/Class2.arff, -S, ESKDB, -K, 5, -I, 5000, -L, 2, -E, 20, -V, -M]

number of label: 14
training time: 1490.

In [44]:
performance = evaluation(y_pred=y_pred_ensemble, y_true=y_true, y_prob=y_prob_ensemble)

# print orders:
print(label.columns[order])

# get confusion matrix
print("\n--- Confusion matrix ---")
get_confusion_matrix(y_pred, y_true, y_pred.columns)

# print performance
print("--- Performance ---")
for key, value in performance.items():
    if key == "f1_each_label":
        print("\n- f1 for each label -")
        for i in range(n_label):
            print("label_"+label.columns[i],"=",round(value[i],2))
    elif key == "coverage_error":
        print(key,'=',round(value,2),"( avg_label_per_instance =",round(avg_label_per_instance,2),")")
    else:
        print(key,'=',round(value,2))

/Users/jiangjunhao/anaconda3/lib/python3.6/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


Index(['Class6', 'Class1', 'Class11', 'Class8', 'Class2', 'Class5', 'Class3',
       'Class4', 'Class7', 'Class13', 'Class9', 'Class10', 'Class14',
       'Class12'],
      dtype='object')

--- Confusion matrix ---
--- Performance ---
coverage_error = 7.53 ( avg_label_per_instance = 4.24 )
ranking_loss = 0.17
hamming_loss = 0.21
f1_macro = 0.36
f1_micro = 0.65
Jaccard_Index = 0.79
zero_one_error = 0.81

- f1 for each label -
label_Class1 = 0.13
label_Class2 = 0.0
label_Class3 = 0.0
label_Class4 = 0.86
label_Class5 = 0.5
label_Class6 = 0.87
label_Class7 = 0.66
label_Class8 = 0.55
label_Class9 = 0.04
label_Class10 = 0.0
label_Class11 = 0.6
label_Class12 = 0.29
label_Class13 = 0.0
label_Class14 = 0.59


# Result

`
**Performance of BR_naive Bayes**
coverage_error = 13.6 ( avg_label_per_instance = 3.25 )
ranking_loss = 0.13
hamming_loss = 0.1
f1_macro = 0.26
f1_micro = 0.4
Jaccard_Index = 0.26
zero_one_error = 0.98
`

`
**Performance of BR_ESKDB**
coverage_error = 14.76 ( avg_label_per_instance = 3.25 )
ranking_loss = 0.15
hamming_loss = 0.08
f1_macro = 0.15
f1_micro = 0.33
jaccard_index = 0.23
zero_one_error = 0.94
`

`
**Performance of Ensemble Classifier Chain using naive Bayes**
coverage_error = 13.63 ( avg_label_per_instance = 3.25 )
ranking_loss = 0.13
hamming_loss = 0.11
f1_macro = 0.26
f1_micro = 0.38
Jaccard_Index = 0.25
zero_one_error = 0.97
`

`
**Performance of Ensemble Classifier Chain using ESKDB(E=2)**
coverage_error = 15.87 ( avg_label_per_instance = 3.25 )
ranking_loss = 0.16
hamming_loss = 0.08
f1_macro = 0.22
f1_micro = 0.38
Jaccard_Index = 0.25
zero_one_error = 0.95
`


recall 体现了分类模型H对正样本的识别能力，recall 越高，说明模型对正样本的识别能力越强.

precision 体现了模型对负样本的区分能力，precision越高，说明模型对负样本的区分能力越强。F1-score 是两者的综合。F1-score 越高，说明分类模型越稳健。